# **Uso de APIs con LlamaIndex**  

Este notebook forma parte del curso de **IA Generativa** de la [Fundación GoodJob](https://www.fundaciongoodjob.org/).  
En él, los alumnos aprenderán los conceptos básicos sobre el uso de **agentes** y cómo integrarlos con **APIs**.  

<p align="center">
  <img src="../notebooks/sources/llamaindex.png" alt="LlamaIndex" style="width:100%; height:auto;"/>
</p>


## Prerrequisitos  

Antes de comenzar con el curso, es necesario leer el documento [setup_instructions.md](../setup_instructions.md),  
donde se detallan los pasos previos indispensables, como:  

- Instalación de **Python**.  
- Instalación de las **dependencias necesarias**.  
- Obtención gratuita de un **token de Hugging Face** como *Inference Provider* para la ejecución de los LLMs en la nube.  
- Acceso gratuito limitado a distintas APIs en tiempo real (e.g. OpenWeather API).  

 


## Login en Hugging Face  

Tras obtener tu **token de acceso** de Hugging Face, realizaremos el inicio de sesión para poder utilizar las **APIs de inferencia serverless**, que se ofrecen de forma gratuita con un límite diario de uso.  

👉 Puedes consultar tu consumo y límites en: [Hugging Face – Inference usage](https://huggingface.co/settings/billing)  


## ¿Qué son los agentes de IA?  

Los **agentes de IA** son sistemas inteligentes que pueden **razonar**, **planificar** y **actuar** de forma autónoma para resolver problemas complejos. A diferencia de un chatbot tradicional que solo responde preguntas, un agente puede:

- 🧠 **Razonar** sobre el problema que necesita resolver
- 🔧 **Utilizar herramientas** externas (APIs, bases de datos, etc.)
- 📋 **Planificar** una secuencia de acciones
- 🔄 **Iterar** hasta encontrar la solución

En este curso aprenderás a crear agentes potentes usando **LlamaIndex**, una de las librerías más populares para desarrollo de aplicaciones de IA.

### ¿Qué vamos a aprender?

1. **Interacción básica** con un modelo de lenguaje
2. **Manejo de contexto** y memoria en conversaciones
3. **Herramientas (Tools)** - cómo dotar a los agentes de superpoderes
4. **Agentes complejos** que pueden consultar APIs en tiempo real

¡Empezamos! 🚀


In [10]:
from huggingface_hub import notebook_login

notebook_login()

## Primeros pasos con agentes

## 1. Tu primer agente básico 🤖

Vamos a empezar con lo más sencillo: crear un agente que pueda mantener una conversación contigo. Este agente será como un asistente virtual que puede entender y responder a tus preguntas.

### Configurando el modelo de lenguaje

Primero, vamos a configurar nuestro modelo de lenguaje usando la API gratuita de Hugging Face:


In [1]:
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

# Configure the language model
llm = HuggingFaceInferenceAPI(
    model_name="HuggingFaceTB/SmolLM3-3B",  # A fast and efficient model
    max_new_tokens=2048,
    temperature=0.2  # Controls creativity: 0=deterministic, 1=very creative
)


print("✅ LLM básico creado correctamente!")


✅ LLM básico creado correctamente!


### ¡Probemos nuestro primer modelo!

Ahora vamos a interactuar con nuestro modelo de lenguaje. Observa cómo puede responder a preguntas básicas:


In [12]:
# Let's chat with our LLM!
def chat_with_agent(message: str, llm = HuggingFaceInferenceAPI(
    model_name="HuggingFaceTB/SmolLM3-3B",  # A fast and efficient model
    max_new_tokens=2048,
    temperature=0.01  # Controls creativity: 0=deterministic, 1=very creative
)):
    """Simple function to chat with our agent"""
    response = llm.complete(message)
    return response


# Test basic conversation
response = chat_with_agent("¡Hola! ¿Cómo estás?")
print("🤖 Agente:", response)

response = chat_with_agent("¿Puedes explicarme qué es la inteligencia artificial en 5 palabras?")
print("🤖 Agente:", response)


🤖 Agente: <think>
Okay, the user sent "¡Hola! ¿Cómo estás?" which is Spanish for "Hello! How are you?" I need to respond in a friendly and welcoming manner. Since the user is using Spanish, I should respond in Spanish as well to keep the conversation consistent. I should ask how they're doing and offer assistance. Let me make sure my response is polite and encourages them to ask questions or share what they need help with. Also, I should keep it concise and natural-sounding. Let me check for any grammar mistakes. "¡Hola! ¿Cómo estás? Me alegra mucho que estés bien. ¿En qué puedo ayudarte hoy?" That sounds good. It's a greeting, a question about their well-being, and an offer to help. I think that's appropriate.
</think>

¡Hola! Me alegra mucho que estés bien. ¿En qué puedo ayudarte hoy?
🤖 Agente: <think>
Okay, the user is asking for a concise explanation of artificial intelligence in five words. Let me start by recalling the key components of AI. It's about machines that can perform ta


### 🔥 La importancia de la temperatura en los LLMs

La **temperatura** es un parámetro fundamental en los modelos de lenguaje (LLMs) que controla la **aleatoriedad** y **creatividad** de las respuestas generadas. 

- Si la temperatura es **baja** (por ejemplo, 0.01), el modelo tiende a ser **más determinista** y predecible, eligiendo casi siempre la palabra más probable.
- Si la temperatura es **alta** (por ejemplo, 1.0), el modelo se vuelve **más creativo** y puede generar respuestas más variadas o inesperadas.

### ¿Cómo funciona la temperatura?

La temperatura ajusta la probabilidad de cada palabra candidata antes de que el modelo elija la siguiente palabra. Matemáticamente, se aplica la siguiente fórmula a las probabilidades originales (*logits*):

$$
p_i' = \frac{e^{\text{logit}_i / T}}{\sum_j e^{\text{logit}_j / T}}
$$

donde:

- **logitᵢ** → puntuación antes de aplicar *softmax*  
- **T** → temperatura  
- **pᵢ'** → probabilidad ajustada de la palabra *i*  

---

- Cuando **T < 1**, las diferencias entre probabilidades se **amplifican** → el modelo es más **determinista**.  
- Cuando **T > 1**, las probabilidades se **suavizan** → el modelo es más **creativo**. 

<center>
  <iframe width="640" height="360" 
  src="https://www.youtube.com/embed/XsLK3tPy9SI" 
  frameborder="0" allowfullscreen></iframe>
</center>





## 2. Añadiendo memoria y contexto : Primer Agente 🧠

¡Genial! Nuestro modelo puede responder preguntas, pero hay un problema: **no recuerda** conversaciones anteriores. Cada pregunta es como si fuera la primera vez que hablamos con él.

Vamos a solucionarlo añadiendo **contexto** para que pueda mantener conversaciones más naturales:


In [9]:
from llama_index.core.workflow import Context
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.llms.openai import OpenAI
import os 
from load_dotenv import load_dotenv

load_dotenv() # Sometimes its a headache, highly recommended using it!
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
llm_openai = OpenAI(
    model="gpt-4o-mini",
    api_key = OPENAI_API_KEY
    )
# Create an agent
simple_agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=[],
    llm=llm_openai,
    system_prompt=(
        "Eres un asistente."
        "Responde siempre en español y brevemente."
    ),
)
# Create a conversation context
ctx = Context(simple_agent)

# Now let's have a conversation with memory!
print("=== Conversación con memoria ===")

# First message - introduce ourselves
response = await simple_agent.run("Hola, mi nombre es María y soy estudiante de ingeniería.", ctx=ctx)
print("👤 Usuario: Hola, mi nombre es María y soy estudiante de ingeniería.")
print("🤖 Agente:", response.response.content)
print()

# Second message - test if it remembers our name
response = await simple_agent.run("¿Recuerdas cuál es mi nombre?", ctx=ctx)
print("👤 Usuario: ¿Recuerdas cuál es mi nombre?")
print("🤖 Agente:", response.response.content)
print()

# Third message - test if it remembers our profession
response = await simple_agent.run("¿Y qué estudio?", ctx=ctx)
print("👤 Usuario: ¿Y qué estudio?")
print("🤖 Agente:", response.response.content)


=== Conversación con memoria ===
👤 Usuario: Hola, mi nombre es María y soy estudiante de ingeniería.
🤖 Agente: ¡Hola, María! ¿En qué área de ingeniería te especializas?

👤 Usuario: ¿Recuerdas cuál es mi nombre?
🤖 Agente: Sí, tu nombre es María.

👤 Usuario: ¿Y qué estudio?
🤖 Agente: Estudias ingeniería.


### ¡Impresionante! 🎉

Como puedes ver, ahora nuestro agente **recuerda** información de mensajes anteriores. Esto es fundamental para crear experiencias de conversación más naturales y útiles.

**¿Cómo funciona?**
- El `Context` mantiene el historial de la conversación
- Cada nuevo mensaje se añade al contexto
- El agente puede "ver" toda la conversación anterior para dar respuestas coherentes


> 💡 **Nota práctica**  
>  
> En nuestras pruebas estamos usando un modelo pequeño de Hugging Face, el **smol 3B**.  
> Este modelo es muy ligero y rápido, lo que lo hace perfecto para **hacer pruebas iniciales** y experimentar con ejemplos sencillos.  
> 
 
Eso sí, conviene tener en cuenta que por su tamaño reducido (unos 3 mil millones de parámetros) a veces **falla en el formato de salida**, se confunde con las instrucciones o genera resultados menos consistentes.  
  
En cambio, cuando utilizamos un modelo más moderno y de mayor escala, como los de **OpenAI (~175B parámetros estimados)**, la experiencia cambia notablemente: obtenemos respuestas mucho más **coherentes, fiables y consistentes**.  
 
👉 La idea es que el modelo pequeño nos sirve como **campo de pruebas rápido**, mientras que los modelos grandes son los que realmente nos permiten **aplicar el flujo en entornos productivos** con mayor confianza.


## 3. Superpoderes para nuestro agente: Las Tools 🔧

Hasta ahora nuestro agente puede conversar y recordar, pero está limitado al conocimiento que tiene el modelo. ¿Qué pasaría si pudiera **calcular**, **buscar información en internet**, o **consultar APIs**?

¡Aquí es donde entran las **Tools** (herramientas)! Las tools permiten que nuestro agente:

- 🧮 **Calcule** operaciones matemáticas exactas
- 🌐 **Acceda** a información en tiempo real
- 📊 **Procese** datos de APIs externas
- 🔍 **Busque** en bases de datos

### Ejemplo simple: Calculadora matemática

Vamos a crear un agente que puede hacer cálculos matemáticos precisos:


In [2]:
from llama_index.core.agent.workflow import ToolCallResult, AgentStream

# Define simple math tools
def sumar(a: int, b: int) -> int:
    """Suma dos números enteros"""
    return a + b

def restar(a: int, b: int) -> int:
    """Resta dos números enteros"""
    return a - b

def multiplicar(a: int, b: int) -> int:
    """Multiplica dos números enteros"""
    return a * b

def dividir(a: float, b: float) -> float:
    """Divide dos números (puede devolver decimales)"""
    if b == 0:
        return "Error: No se puede dividir por cero"
    return a / b

def potencia(a:float, power:int):
    """Eleva un número a la n-ésima potencia"""
    if power == 0 : 
        return 1
    return a**power

# Create an agent with math tools
math_agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=[sumar, restar, multiplicar, dividir, potencia],
    llm=llm_openai,
    system_prompt=(
        "Eres un asistente matemático que puede realizar cálculos exactos. "
        "Utiliza las herramientas disponibles ( [sumar, restar, multiplicar, dividir, potencia] ) para hacer cálculos precisos. "
        "Responde siempre en español y explica brevemente qué operación realizaste."
    ),
)
print(f"🧮 Agente matemático creado con tools: sumar, restar, multiplicar, dividir")

# Create an dumb agent without math tools
dumb_math_agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=[],
    llm=llm_openai,
    system_prompt=(
        "Eres un asistente matemático que puede realizar cálculos exactos. "
        "Responde siempre en español y explica brevemente qué operación realizaste."
    ),
)
print("🧮 Agente pseudo-matemático creado sin tools")



🧮 Agente matemático creado con tools: sumar, restar, multiplicar, dividir
🧮 Agente pseudo-matemático creado sin tools


### ¡Probemos las herramientas matemáticas!

Ahora vamos a ver cómo nuestro agente con herramientas puede **razonar** y **decidir** qué herramienta usar para resolver problemas matemáticos, mientras que nuestro agente simple es incapaz de resolver operaciones matemáticas complejas:


In [4]:
# Function to test math agent with streaming
async def test_math_agent(question: str, dumb = False):
    print(f"👤 Usuario: {question}")
    print("🤖 Agente: ", end="", flush=True)
    agent = dumb_math_agent if dumb else math_agent
    handler = agent.run(question)
    
    # Stream the response and show tool calls
    async for ev in handler.stream_events():
        if isinstance(ev, ToolCallResult):
            print(f"\n🔧 Usando herramienta: {ev.tool_name} con parámetros {ev.tool_kwargs}")
            print(f"📊 Resultado: {ev.tool_output}")
            print("🤖 Agente: ", end="", flush=True)
        elif isinstance(ev, AgentStream):
            print(ev.delta, end="", flush=True)
    
    response = await handler
    print("\n" + "="*50)
    return response

# Test various math problems
await test_math_agent("What is (2 + 2) ** 65? Give me just the result")
await test_math_agent("What is (2 + 2) ** 65? Give me just the result", dumb = True)
# await test_math_agent("Si tengo 100 euros y gasto 23, ¿cuánto me queda?")
# await test_math_agent("¿Cuál es el resultado de (8 * 5) / 2?")


👤 Usuario: What is (2 + 2) ** 65? Give me just the result
🤖 Agente: 
🔧 Usando herramienta: potencia con parámetros {'a': 4, 'power': 65}
📊 Resultado: 1361129467683753853853498429727072845824
🤖 Agente: El resultado de \( (2 + 2)^{65} \) es 1361129467683753853853498429727072845824.
👤 Usuario: What is (2 + 2) ** 65? Give me just the result
🤖 Agente: El resultado de \( (2 + 2)^{65} \) es \( 4^{65} \), que es igual a \( 1,125,899,906,842,624 \). 

Realicé la operación sumando 2 + 2 para obtener 4 y luego elevé 4 a la potencia de 65.


AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='El resultado de \\( (2 + 2)^{65} \\) es \\( 4^{65} \\), que es igual a \\( 1,125,899,906,842,624 \\). \n\nRealicé la operación sumando 2 + 2 para obtener 4 y luego elevé 4 a la potencia de 65.')]), structured_response=None, current_agent_name='Agent', raw={'id': 'chatcmpl-C9Ihe3Lr4QBDj5pwNwOy0FXcnkd2r', 'choices': [{'delta': {'content': None, 'function_call': None, 'refusal': None, 'role': None, 'tool_calls': None}, 'finish_reason': 'stop', 'index': 0, 'logprobs': None}], 'created': 1756332882, 'model': 'gpt-4o-mini-2024-07-18', 'object': 'chat.completion.chunk', 'service_tier': 'default', 'system_fingerprint': 'fp_560af6e559', 'usage': None, 'obfuscation': 'nq'}, tool_calls=[], retry_messages=[])

### ¡Increíble! 🤯

¿Te das cuenta de lo que acaba de pasar? Nuestro agente:

1. **Analizó** la pregunta matemática
2. **Decidió** qué herramienta necesitaba usar
3. **Ejecutó** la herramienta con los parámetros correctos
4. **Interpretó** el resultado y lo presentó de forma amigable

Esto es **razonamiento automático** en acción. El agente no solo ejecuta código, sino que **piensa** sobre qué hacer y cómo hacerlo.


## 4. Agente avanzado: Consultando el tiempo en tiempo real 🌤️

Ahora vamos a crear algo realmente emocionante: un agente que puede consultar el **tiempo actual** de cualquier ciudad del mundo usando una API externa.

Este ejemplo demuestra cómo los agentes pueden:
- 🌍 Acceder a **datos en tiempo real**
- 🔄 Mantener **contexto** en conversaciones largas
- 🧠 **Combinar** múltiples habilidades (conversación + herramientas)

### Configurando la herramienta del tiempo

Vamos a usar la herramienta que ya tenemos creada en nuestro proyecto:


In [3]:
import sys
import os
from dotenv import load_dotenv

# Add src to path to import our weather tool
sys.path.append(os.path.join(os.getcwd(), '..'))

# Load environment variables
load_dotenv()

# Import our weather tool
from src.apis.weather_api import current_weather_tool

# Create the weather tool
weather_tool = current_weather_tool()

# Create an advanced agent with weather capabilities
weather_agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=[weather_tool],
    llm=llm_openai,
    system_prompt=(
        "Eres un asistente meteorológico inteligente que puede consultar el tiempo actual "
        "de cualquier ciudad del mundo. Siempre responde en español de manera amigable y "
        "proporciona información útil sobre el clima. Si no entiendes una ciudad, "
        "pregunta por aclaración."
    ),
)

print("🌤️ Agente meteorológico creado con acceso a datos del tiempo en tiempo real!")


🌤️ Agente meteorológico creado con acceso a datos del tiempo en tiempo real!


### ¡Probemos nuestro agente meteorológico con contexto!

Ahora vamos a tener una conversación completa con nuestro agente, donde mantenga el contexto y pueda acceder a datos del tiempo en tiempo real:


In [4]:
# Create conversation context for weather agent
weather_ctx = Context(weather_agent)

async def conversation_with_weather_agent(message: str, ctx):
    """Chat with weather agent showing the complete process"""
    print(f"👤 Usuario: {message}")
    print("🤖 Agente: ", end="", flush=True)
    
    handler = weather_agent.run(message, ctx=ctx)
    
    # Stream the response and show tool calls
    async for ev in handler.stream_events():
        if isinstance(ev, ToolCallResult):
            print(f"\n🔧 Consultando API del tiempo...")
            print(f"📡 Parámetros: {ev.tool_kwargs}")
            print(f"📊 Datos recibidos: {ev.tool_output}")
            print("🤖 Agente: ", end="", flush=True)
        elif isinstance(ev, AgentStream):
            print(ev.delta, end="", flush=True)
    
    response = await handler
    print("\n" + "="*60)
    return response

# Have a complete conversation with context and weather tools
print("🌤️ Iniciando conversación con el agente meteorológico...\n")

await conversation_with_weather_agent(
    "Hola, me llamo Carlos y vivo en Madrid. ¿Podrías decirme qué tiempo hace hoy aquí?", 
    weather_ctx
)

await conversation_with_weather_agent(
    "¿Y qué tal está el tiempo en Barcelona?", 
    weather_ctx
)

await conversation_with_weather_agent(
    "¿Recuerdas dónde vivo? ¿Crees que debería llevar paraguas hoy?", 
    weather_ctx
)


🌤️ Iniciando conversación con el agente meteorológico...

👤 Usuario: Hola, me llamo Carlos y vivo en Madrid. ¿Podrías decirme qué tiempo hace hoy aquí?
🤖 Agente: 
🔧 Consultando API del tiempo...
📡 Parámetros: {'city': 'Madrid,ES'}
📊 Datos recibidos: city='Madrid' country='ES' condition='clear sky' temperature=24.72 feels_like=24.0 humidity=29 wind_speed=5.66 units='metric'
🤖 Agente: ¡Hola, Carlos! Hoy en Madrid el clima es bastante agradable. Actualmente, la temperatura es de aproximadamente 24.7 °C, con una sensación térmica de 24.0 °C. La humedad está en un 29%, y hay un viento suave de 5.66 km/h. ¡Disfruta del día!
👤 Usuario: ¿Y qué tal está el tiempo en Barcelona?
🤖 Agente: 
🔧 Consultando API del tiempo...
📡 Parámetros: {'city': 'Barcelona,ES'}
📊 Datos recibidos: city='Barcelona' country='ES' condition='light rain' temperature=25.84 feels_like=26.2 humidity=66 wind_speed=4.63 units='metric'
🤖 Agente: En Barcelona, el clima es un poco diferente. Actualmente, hay una ligera lluvia y 

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='Sí, recuerdo que vives en Madrid. Hoy en Madrid el clima es soleado y no hay lluvia, así que no necesitas llevar paraguas. ¡Disfruta de tu día al aire libre! Si necesitas más información, no dudes en preguntar.')]), structured_response=None, current_agent_name='Agent', raw={'id': 'chatcmpl-C9ImQRRhqVoImUdjhsJ8JTbDbrKNr', 'choices': [{'delta': {'content': None, 'function_call': None, 'refusal': None, 'role': None, 'tool_calls': None}, 'finish_reason': 'stop', 'index': 0, 'logprobs': None}], 'created': 1756333178, 'model': 'gpt-4o-mini-2024-07-18', 'object': 'chat.completion.chunk', 'service_tier': 'default', 'system_fingerprint': 'fp_560af6e559', 'usage': None, 'obfuscation': 'E5'}, tool_calls=[], retry_messages=[])

## ¡Felicidades! 🎉 Has creado tu primer agente inteligente

### ¿Qué hemos logrado?

En este notebook hemos creado agentes progresivamente más sofisticados:

1. **🤖 Agente básico**: Conversación simple sin herramientas
2. **🧠 Agente con memoria**: Mantiene contexto entre mensajes
3. **🔧 Agente con tools**: Puede ejecutar herramientas matemáticas
4. **🌍 Agente avanzado**: Accede a APIs externas en tiempo real + contexto

### ¿Por qué es esto revolucionario?

- **Razonamiento automático**: El agente decide qué hacer y cuándo
- **Acceso a datos externos**: No limitado al conocimiento pre-entrenado
- **Conversaciones naturales**: Mantiene el contexto como un humano
- **Extensibilidad**: Puedes añadir cualquier API o herramienta

### ¿Qué sigue?

En los próximos notebooks aprenderás:

- ✅ ✨ **Introducción a los LLMs, agentes y tools**
- ⬜ 📰 **Agentes que buscan noticias**
- ⬜ 📄 **Agentes que procesan documentos**
- ⬜ 🏗️ **Agentes multi-modales más complejos**
- ⬜ 🚀 **Despliegue de agentes en producción**

¡El mundo de los agentes de IA está apenas comenzando! 🌟
